In [229]:
import pandas as pd
import requests
import praw
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from datetime import datetime, timedelta
import pytz

In [230]:
sp = pd.read_csv("sp500_2019.csv")
sp['Open'] = sp['Open'].str.replace(',', '')
sp['Close'] = sp['Close'].str.replace(',', '')
sp['Open'] = pd.to_numeric(sp['Open'])
sp['Close'] = pd.to_numeric(sp['Close'])
sp['DayResult'] = sp['Open'] - sp['Close']
sp['Result'] = sp['DayResult'].map(lambda x: x >= 0)

def formatDate(originalDate):
    ans = ""
    parts = originalDate.split("/")
    ans += parts[2]
    ans += "-"
    ans += parts[0]
    ans += "-"
    ans += parts[1]
    return ans  

sp['Date'] = sp["Date"].apply(lambda date: formatDate(date))
sp['Date'] = sp['Date'].values[::-1]
sp.head(3)
#print(sp.shape)

,Date,Open,High,Low,Close,DayResult,Result
0,2019-01-02,3215.18,"3,231.72","3,212.03",3230.78,-15.60,False
1,2019-01-03,3240.09,"3,240.92","3,216.57",3221.29,18.80,True
2,2019-01-04,3247.23,"3,247.93","3,234.37",3240.02,7.21,True


In [231]:
news = pd.read_csv("top25news2019.csv")
#news = news.drop(news.columns[0], axis=1)
stockDaysList = sp['Date'].tolist()
news['Active'] = news['Date'].apply(lambda date: date in stockDaysList)
news = news[news['Active'] == True]
news.reset_index(inplace=True)
news = news.drop(news.columns[0], axis=1)
news = news.drop(news.columns[26], axis=1)
print(news.shape)

(255, 26)


In [232]:
news['Result'] = sp['Result'].astype(int)
cols = news.columns.tolist()
cols.insert(1, cols[-1])
cols = cols[:-1]
news = news[cols]


In [233]:
news = news.fillna(5)
news = news.astype({"Result": int})
news.to_csv('completeNewsSet2019.csv', index=False)
news.head()

,Date,Result,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2019-01-02,0,Former Blackwater guard convicted for 2007 mas...,Chinese Navy ship seen carrying a railgun capa...,Mexican mayor gunned down after being sworn in,Italian health czar resigns over government's ...,Average 10-year-old has eaten 18 years' worth ...,Brazil’s newly inaugurated President Jair Bols...,A '620km human chain' — Indian Women Rally for...,"Hours after taking office, Brazil’s new presid...",CIA-managed forces in Afghanistan are reported...,"UK Shoppers have earned more than £30,000 by r...",Almost a third of new cars sold in Norway last...,"Xi Jinping says Taiwan ""must and will be"" reun...",Abused Korean girl’ statue removed after Japan...,Ukrainian Embassy calls out CNN for calling th...,Taiwan's President Tsai Ing-wen said on Wednes...,"Indian women enter Hindu temple, defying centu...",Deadly train crash on Danish bridge,Vietnam criticised for 'totalitarian' law bann...,Trump signs pro-Taiwan 'Asia Reassurance Initi...,Bill Gates' nuclear venture hits snag amid U.S...,Russian Billionaire Demolishes Trump's Former ...,Tens of thousands of people gathered in Tokyo ...,Hundreds of thousands of women have formed a 6...,Netflix Drops Hasan Minhaj Episode In Saudi Ar...,Nasa's New Horizons: 'Snowman' shape of distan...
1,2019-01-03,1,Hasan Minhaj responds after Netflix pulls epis...,Google shifted $23 billion to tax haven Bermud...,Saudi Arabia opens Khashoggi murder trial,U.S. President Donald Trump on Wednesday mocke...,Israeli official 'in shock' as Trump says Iran...,US government warns Americans that China may n...,Bolivia Rolls Out Universal Health Care To 5.8...,Maggi ban: Nestle admits presence of lead in p...,It looks a lot like North Korea’s ambassador t...,First women to enter India temple in centuries...,N Korea ambassador to Italy 'disappears',Mexico finds Flayed god temple; priests wore s...,Lesbian couple to wed 26 times abroad to seek ...,Video games sector now accounts for more than ...,Investigators have uncovered another illicit t...,India reminds U.S. that it sends soldiers abro...,Two gay men mount first legal challenges to Ho...,Stabbings at Sydney Scientology centre leave o...,Google shifted $23 billion to tax haven Bermud...,The president of Italy's national health resea...,Trump says he could be 'most popular person in...,"Cargo Ship Carrying 3,500 Nissan Cars Ablaze a...",Berlin nightclubs are turning into overnight s...,Mexican experts have found the first temple of...,Russia Announces Plans To Set Up Its First Eve...
2,2019-01-04,1,Workers at Amazon’s biggest warehouse in Spain...,Trump defends Soviet Union's 1979 invasion of ...,Navy SEAL To Plead Not Guilty To War Crimes - ...,"Since Huawei executive's arrest, 13 Canadians ...",Renewable energy became Germany's dominant sou...,Egyptian President Tells 60 Minutes He Collabo...,Germans happiest to pay taxes to fund infrastr...,Hackers have released confidential data linked...,China's population shrinks for first time in 7...,One of the most senior German figures in the C...,Mexican President Andres Manuel Lopez Obrador ...,UK health body calls for Copenhagen-style bike...,A suspected case of Ebola has been reported at...,The bosses of the UK’s biggest companies are f...,"Venezuela's Hyperinflation Hits 80,000% Per Ye...",Former Philippine mayor killed hours after Pre...,Kuwait rejected to co-host the 2022 FIFA World...,Trump curiously well versed in specific Russia...,Pope Francis’ Argentinean Protegé Accused of S...,'Mass hack attack' on German politicians,"A German far-right party, the NPD, has threate...",Explosion outside AfD office in eastern Germany,Brazil's Bolsonaro begins starts firing 'left-...,Heatwave conditions predicted to affect Austra...,Brazilian President Jair Bolsonaro—whose inaug...
3,2019-01-07,0,Former Venezuela Supreme Court judge flees to ...,White House n